# Tratamento dos dados 

## Bibliotecas

In [1]:
import pandas as pd 
import numpy as np 
import scipy 
import _pickle as cPickle
from sklearn import model_selection, preprocessing, feature_selection, ensemble, linear_model, metrics, decomposition

## Importando os dados

In [2]:
df = pd.read_csv('pov19.csv')

## Limpando os dados

### Excluindo colunas

In [3]:
df = df.drop(['Unnamed: 0', 'pobre_ex'], axis = 1)

### Eliminar os missings values

In [4]:
df = df.dropna(subset=['esgoto_ad'])

### Definindo listas de nomes

#### Definição de uma lista com nomes das variáveis categoricas

In [5]:
cat_vars = ['area', 'casal', 'genero', 'raca', 'escolaridade', 'atividade', 'aposentadoria', 
            'dom_tipo', 'agua_adeq', 'esgoto_ad', 'banheiro', 'paredes', 'casa_tipo', 
            'geladeira', 'maquina_lavar', 'tv', 'computador', 'internet', 'carro', 'moto']

#### Definição de uma lista com nomes das variáveis numéricas


In [6]:
num_vars = ['hh_id','idade', 'n_moradores', 'n_criancas', 'n_adultos', 'densidade', 'fone_pc']

### Filtrando colunas 


In [7]:
df = df[['hh_id','pobre', 'area', 'casal', 'genero', 'raca', 'escolaridade', 'atividade', 'aposentadoria', 
                   'dom_tipo', 'agua_adeq', 'esgoto_ad', 'banheiro', 'paredes', 'casa_tipo', 
                   'geladeira', 'maquina_lavar', 'tv', 'computador', 'internet', 'carro', 'moto', 
                   'idade', 'n_moradores', 'n_criancas', 'n_adultos', 'densidade', 'fone_pc']]

### Criando dummy para a variavel e resposta

In [8]:
df['pobre'] = pd.get_dummies(df.pobre, drop_first=True)


### Criando Dummy para Regiões

#### Nordeste

In [9]:
nordeste = []
for cod in df['hh_id']:
    match int(cod/100000000000):
        case 21 | 22 | 23 | 24 | 25 | 26 | 27 | 28 | 29 :
            nordeste.append(1)
        case _:
            nordeste.append(0)
df['Nordeste'] = nordeste

#### Norte

In [10]:
norte = []
for cod in df['hh_id']:
    match int(cod/100000000000):
        case 11 | 12 | 13 | 14 | 15 | 16 | 17:
            norte.append(1)
        case _:
            norte.append(0)
df['Norte'] = norte

#### Sudeste 

In [11]:
sudeste = [] 
for cod in df['hh_id']:
    match int(cod/100000000000):
        case 31 | 32 | 33 | 34:
            sudeste.append(1)
        case _:
            sudeste.append(0)
df['Sudeste'] = sudeste 

#### Sul 

In [12]:
sul = [] 
for cod in df['hh_id']:
    match int(cod/100000000000):
        case 41 | 42 | 43 :
            sul.append(1)
        case _:
            sul.append(0)
df['Sul'] = sul

## Exportando os dados Limpos 

In [13]:
df.to_csv("pov19ce_clean.csv")

## Pre Tratamento 

### Divisão da amostra 

In [14]:
df_train, df_test = model_selection.train_test_split(df, test_size=0.25, stratify= df.pobre, random_state=1)

### Ajustar os dados de treino

In [15]:
X_train = df_train.drop('pobre', axis=1)
y_train = df_train['pobre']
X_test = df_test.drop('pobre',axis=1)
y_test = df_test['pobre']

### Normalizar os dados 

In [16]:
X_train

,hh_id,area,casal,genero,raca,escolaridade,atividade,aposentadoria,dom_tipo,agua_adeq,...,idade,n_moradores,n_criancas,n_adultos,densidade,fone_pc,Nordeste,Norte,Sudeste,Sul
10412,1500318200608,urbano,casado,mulher,branco,superior completo,ativo ocupado,nao,casa,adequada,...,50,4,0,4,0.666667,1.000000,0,1,0,0
57424,2800304601108,urbano,solteiro,homem,preto ou pardo,sem instrucao,inativo,nao,casa,adequada,...,80,1,0,1,0.166667,0.000000,1,0,0,0
46529,2600228141008,urbano,casado,mulher,preto ou pardo,medio completo,ativo ocupado,nao,casa,adequada,...,40,2,0,2,0.333333,1.000000,1,0,0,0
69558,3100823670207,urbano,casado,homem,preto ou pardo,fundam incompleto,ativo ocupado,nao,casa,adequada,...,58,4,0,4,1.000000,1.000000,0,0,1,0
71163,3100676560608,urbano,casado,homem,branco,medio completo,inativo,sim,casa,adequada,...,69,3,0,3,0.600000,1.000000,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124028,4200730361107,urbano,solteiro,mulher,branco,fundam incompleto,inativo,sim,casa,adequada,...,83,1,0,1,0.166667,0.000000,0,0,0,1
110725,4101174890707,urbano,solteiro,homem,branco,superior completo,ativo desocupado,nao,casa,adequada,...,29,7,1,6,1.000000,1.000000,0,0,0,1
142641,5100643111008,rural,casado,homem,branco,fundam incompleto,ativo ocupado,nao,casa,adequada,...,37,4,1,3,0.500000,1.333333,0,0,0,0
13475,1500844160708,urbano,solteiro,mulher,preto ou pardo,sem instrucao,inativo,sim,casa,inadequada,...,62,1,0,1,0.200000,1.000000,0,1,0,0


In [17]:
scaler = preprocessing.MinMaxScaler(feature_range=(0,1))
X_train[num_vars] = scaler.fit_transform(X_train[num_vars])
X_test[num_vars] = scaler.fit_transform(X_test[num_vars])


### Criar dummies para variáveis categóricas

In [18]:
cat_vars = ['area', 'casal', 'genero', 'raca', 'escolaridade', 'atividade', 'aposentadoria', 
            'dom_tipo', 'agua_adeq', 'esgoto_ad', 'banheiro', 'paredes', 'casa_tipo', 
            'geladeira', 'maquina_lavar', 'tv', 'computador', 'internet', 'carro', 'moto']
X_train = pd.get_dummies(data = X_train, columns = cat_vars)
X_test = pd.get_dummies(data = X_test, columns = cat_vars)

## Selecionando os Atributos

### Definir atributos e variável de resposta no conjunto de treino

In [19]:
X = X_train.values
y = df_train['pobre'].values
feature_names = X_train.columns 

### Teste Anova

In [20]:
selector = feature_selection.SelectKBest(score_func=  
               feature_selection.f_classif, k=15).fit(X,y)
anova_selected_features = feature_names[selector.get_support()]


### Lasso

In [21]:
selector = feature_selection.SelectFromModel(estimator= 
              linear_model.LogisticRegression(C=1, penalty="l1", 
              solver='liblinear'), max_features=15).fit(X,y)
lasso_selected_features = feature_names[selector.get_support()]

### Relevância de atributos 

In [22]:
feature_names = X_train.columns.tolist()

### Importância dos atributos

In [23]:
model = ensemble.RandomForestClassifier(n_estimators=100,
                      criterion="entropy", random_state=0)
model.fit(X,y)
importances = model.feature_importances_

In [24]:
df_importances = pd.DataFrame({"IMPORTANCE":importances, "VARIABLE":feature_names}).sort_values("IMPORTANCE", ascending=False)
df_importances['cumsum'] = df_importances['IMPORTANCE'].cumsum(axis=0)
df_importances = df_importances.set_index("VARIABLE")

## Exportando os dataframes tratados

### Criando dicionario com valores exportados

In [25]:
output = {
    'X_train':X_train,
    'X_test':X_test,
    'y_train':y_train,
    'y_test':y_test,
    'dataframe':df,
    'Anova_features':anova_selected_features,
    'Lasso_features':lasso_selected_features,
    'Random_Forest':model,
    'Importances':df_importances
}

### Exportando os outputs

In [26]:
with open(r"poverty.pickle", "wb") as output_file:
   ...:     cPickle.dump(output, output_file)